<a href="https://colab.research.google.com/github/leman-cap13/NLP_projects/blob/main/Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import get_dataset_config_names

domains=get_dataset_config_names("subjqa")
domains

In [ ]:
!pip install datasets --upgrade

In [ ]:
from datasets import load_dataset

subjqa=load_dataset("subjqa", name='electronics')
subjqa

In [ ]:
#open domain and close-domain(spesific)

In [ ]:
print(subjqa['train']['answers'][1])

In [ ]:
print(subjqa['train'][1])

In [ ]:
import pandas as pd

dfs={split : dset.to_pandas() for split, dset in subjqa.flatten().items()}

for split, df in dfs.items():
  print(f"Numner of questions in { split}: { df['id'].nunique()}")

In [ ]:
qa_cols=['title','question', 'answers.text', 'answers.answer_start', 'context']
sample_df=dfs['train'][qa_cols].sample(2, random_state=7)
sample_df

In [ ]:
start_idx=sample_df['answers.answer_start'].iloc[0][0]
end_idx=start_idx+len(sample_df['answers.text'].iloc[0][0])
sample_df['context'].iloc[0][start_idx:end_idx]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
counts={}
question_types=['What','How','Is','Does','Do','Was','Where','Why']

for q in question_types:
    counts[q]=dfs['train']['question'].str.startswith(q).value_counts()[True]

pd.Series(counts).sort_values().plot.barh()
plt.title('Frequency of Question Types')
plt.show()



In [ ]:
for question_type in ['How','What','Is']:
  for question in (
      dfs['train'][dfs['train'].question.str.startswith(question_type)]
      .sample(n=3, random_state=42)['question']):
      print(question)


#Tokenizing text for QA

In [ ]:
from transformers import AutoTokenizer

model_ckpt='deepset/minilm-uncased-squad2'
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
#minilm

In [ ]:
question='How much music can this hold?'
context='An MP3 is about 1 MB/minute, so about 6000 hours depending on file size'
inputs=tokenizer(question, context, return_tensors='pt')

In [ ]:
input_df=pd.DataFrame.from_dict(tokenizer(question, context), orient='index')
input_df

In [ ]:
print(tokenizer.decode(inputs['input_ids'][0]))

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

model=AutoModelForQuestionAnswering.from_pretrained(model_ckpt)


with torch.no_grad():
  outputs=model(**inputs)

print(outputs)

In [ ]:
import numpy as np

In [ ]:
s_scores=outputs.start_logits.detach().numpy().flatten()
e_scores=outputs.end_logits.detach().numpy().flatten()
tokens=tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy()[0])

fig, (ax1,ax2)=plt.subplots(nrows=2,sharex=True, figsize=(10,4))
colors=['C0' if s!=np.max(s_scores) else 'C1' for s in s_scores]
ax1.bar(x=tokens, height=s_scores, color=colors)
ax1.set_ylabel('Start Scores')
colors=['C0' if e!=np.max(e_scores) else 'C1' for e in e_scores]
ax2.bar(x=tokens, height=e_scores, color=colors)
ax2.set_ylabel('End Scores')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
import torch

start_logits=outputs.start_logits
end_logits=outputs.end_logits

start_idx=torch.argmax(start_logits)
end_idx=torch.argmax(end_logits)
answer_span=inputs['input_ids'][0][start_idx: end_idx]
answer=tokenizer.decode(answer_span)
print(f"Question: {question}")
print(f"Answer: {answer}")

In [ ]:
#extractive only encoder, anstractive encoder- decoder modellerden istifade edir

In [ ]:
from transformers import pipeline

pipe=pipeline('question-answering', model=model, tokenizer=tokenizer)
pipe(question=question, context=context, topk=3)

In [ ]:
#what if we dont have an answer?

pipe(question='Why is there no data?', context=context, handle_impossible_answer=True)

In [ ]:
#if answer pass context size(512 tokens)
#sliding window ve ya logformer bigbird kimi modeller

#Dealing with long passages

In [ ]:
def compute_input_length(row):
  inputs=tokenizer(row['question'], row['context'])
  return len(inputs['input_ids'])
dfs['train']['n_tokens']=dfs['train'].apply(compute_input_length, axis=1)

fig, ax=plt.subplots()
dfs['train']['n_tokens'].hist(bins=100, grid=False, ax=ax)
plt.xlabel('Number of tokens in question-context pair')
ax.axvline(x=512, ymin=0, ymax=1, linestyle='--',
           color='C1', label='Maximum sequence length')

plt.legend()
plt.ylabel('Count')
plt.show()

In [ ]:
example=dfs['train'].iloc[0][['question','context']]
tokenized_example=tokenizer(example['question'],example['context'],
                            return_overflowing_tokens=True,max_length=100,
                            stride=25,truncation=True)


In [ ]:
for idx, window in enumerate(tokenized_example['input_ids']):
  print(f"window #{idx} has {len(window)} tokens")

In [ ]:
for window in tokenized_example['input_ids']:
  print(f'{tokenizer.decode(window)}\n')

#Using Haystack to build a QA Pipeline

In [ ]:
url = """https://artifacts.elastic.co/downloads/elasticsearch/\
elasticsearch-7.9.2-linux-x86_64.tar.gz"""
!wget -nc -q {url}
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT

# Run Elasticsearch as a background process
!chown -R daemon:daemon elasticsearch-7.9.2
es_server = Popen(args=['elasticsearch-7.9.2/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))
# Wait until Elasticsearch has started
!sleep 30

In [ ]:
#Popen ? arxa planda run etmek

In [ ]:
!curl -X GET 'localhost:9200/?pretty'

In [ ]:
!pip install  datasets --upgrade

In [ ]:
!pip install farm-haystack[elasticsearch]

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore



In [ ]:
document_store=ElasticsearchDocumentStore(
    host='localhost',
    port=9200,
    username='',
    password='',
    index='document',
    return_embedding=True
)


In [ ]:
for split , df in dfs.items():
  docs=[{'content':row['context'],
         'meta':{'item_id':row['title'], 'question_id': row['id'],
                 'split': split}}
        for _,row in df.drop_duplicates(subset='context').iterrows()]
  document_store.write_documents(docs,index='document')
print(f"Loaded { document_store.get_document_count()} documents")


In [ ]:
# cunki retrievel xussui format gozleyir content meta

#Initializing a retriever

In [ ]:
from haystack.nodes import BM25Retriever

retriever=BM25Retriever(document_store=document_store)

In [ ]:
item_id='B0074BW614'
querry='Is it good for reading?'
retriever_docs=retriever.retrieve(query=querry, top_k=3,
                                  filters={'item_id':[item_id],
                                           'split':['train']})

In [ ]:
retriever_docs[0].content

In [ ]:
retriever_docs[1].content

#Initializing a reader

In [ ]:
from haystack.nodes import TransformersReader

model_ckpt='deepset/minilm-uncased-squad2'
reader=TransformersReader(
    model_name_or_path=model_ckpt,
    max_seq_len=384,
    doc_stride=128
)

#Putting it all together

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe=ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [ ]:
n_answers=3
preds=pipe.run(
    query=querry,
    params={
        'Retriever':{'top_k':3,
                     'filters':{'item_id':[item_id],
                                'split': ['train']}},
        'Reader' : { 'top_k': n_answers}
    }
)

In [ ]:
print(f"Qustion: { preds['query']}\n")
answers=preds.get('answers',[])

if not answers:
  print('No answers found')
else:
  for idx,ans in enumerate(answers):
    print(f"Answer {idx+1}: {ans.answer}")
    print(f"Review snippet:...{ans.context}...\n")
